# Observation noise


What do you do if your function evaluations are noisy?
Plain finite difference schemes struggle in this setup.

In [1]:
import jax.numpy as jnp
from probfindiff import central, differentiate
from jax import random

key = random.PRNGKey(seed=1)

Noisy observations (even with very little noise) tend to mess up finite difference approximations. 

In [2]:
scheme, xs = central(dx=0.01)
noise = 1e-2 * random.normal(key, shape=xs.shape)

df, _ = differentiate(jnp.sin(xs) + noise, scheme=scheme)
df, jnp.cos(0.0)

(DeviceArray(1.5257977, dtype=float32),
 DeviceArray(1., dtype=float32, weak_type=True))

Once we know which kind of error we are dealing with (noisy function evaluations), we can tune the FD scheme accordingly.

In [3]:
scheme, xs = central(dx=0.01, noise_variance=1e-4)
df, _ = differentiate(jnp.sin(xs) + noise, scheme=scheme)
df, jnp.cos(0.0)

(DeviceArray(1.0176924, dtype=float32),
 DeviceArray(1., dtype=float32, weak_type=True))

Mathematically, this is due to the fact that PN finite differences do some clever Gaussian process regression, which deals well with noise.
The takeaway is: don't overfit your finite difference scheme. Adapt it to incorporate noise.